In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the zip files in Google Drive
model_zip_path = '/content/drive/MyDrive/bert_suicide_model.zip'
tokenizer_zip_path = '/content/drive/MyDrive/tokenizer (1).zip'

# Directory to unzip the files
model_dir = '/content/bert_suicide_model'
tokenizer_dir = '/content/tokenizer (1)'

# Unzip the files
with zipfile.ZipFile(model_zip_path, 'r') as zip_ref:
    zip_ref.extractall(model_dir)

with zipfile.ZipFile(tokenizer_zip_path, 'r') as zip_ref:
    zip_ref.extractall(tokenizer_dir)

print("Model and tokenizer unzipped successfully!")


Model and tokenizer unzipped successfully!


In [ ]:


import tensorflow as tf
from transformers import AutoTokenizer

# Path to the unzipped model and tokenizer in Colab
model_name = '/content/bert_suicide_model'  # Path to the unzipped model directory
tokenizer_name = '/content/tokenizer (1)'  # Path to the unzipped tokenizer directory

# Load the TensorFlow model
model = tf.saved_model.load(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)


print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [ ]:
# making predictions.
def test_suicide_detection(texts, model, tokenizer):
    """
    Predicts whether input texts indicate suicidal thoughts or not.

    Args:
        texts (list): List of text strings to classify.
        model: Trained BERT classifier model.
        tokenizer: Tokenizer used during training.

    Returns:
        None. Prints out the text and its predicted category (suicide/non-suicide).
    """
    label_mapping = {0: "non-suicide", 1: "suicide"}

    # Tokenize input text, including token_type_ids, and ensuring padding to the model's expected length
    inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=128, return_tensors="tf", return_token_type_ids=True) # Changed padding to 'max_length'

    # Cast input tensors to tf.int64
    inputs = {key: tf.cast(value, tf.int64) for key, value in inputs.items()}

    # Calling function for inference
    infer = model.signatures['serving_default']  # The inference function
    outputs = infer(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], token_type_ids=inputs['token_type_ids']) # Added token_type_ids

    # Get predictions (logits)
    logits = outputs['output_1']
    predictions = tf.argmax(logits, axis=1).numpy()

    # Map predictions to labels
    predicted_labels = [label_mapping[pred] for pred in predictions]

    # Print results
    print("\nPredicted Suicide Classification:")
    for text, label in zip(texts, predicted_labels):
        print(f"Text: '{text}' => Predicted Category: {label}")

# Example texts to classify
test_texts = [
    "I don’t think I can do this anymore, I just want everything to stop.",
    "No one would even care if I disappeared.",
    "I feel so empty, like I don’t even exist.",
    "I just want to sleep forever and never wake up.",
    "Everything is falling apart, I can’t take it anymore.",
    "I keep thinking about ways to make the pain stop.",
    "I’ve lost all hope, nothing matters anymore.",
    "It feels like the whole world is moving on without me.",
    "I had a great time with my friends today, feeling really happy!",
    "Im soooo proud about my self",
    "I failed my test today, but I will try harder next time.",
    "My freinds are so hillarious and I love spending time with them",
    "I love going out with her, my heart is getting filled with love",
    "I'm excited to start my new job next week!",
    "I can't wait for the weekend, so many fun things planned!",
    "What a great game today wass.. Cant believe we won"
]

# Run the test
test_suicide_detection(test_texts, model, tokenizer)


Predicted Suicide Classification:
Text: 'I don’t think I can do this anymore, I just want everything to stop.' => Predicted Category: suicide
Text: 'No one would even care if I disappeared.' => Predicted Category: suicide
Text: 'I feel so empty, like I don’t even exist.' => Predicted Category: suicide
Text: 'I just want to sleep forever and never wake up.' => Predicted Category: suicide
Text: 'Everything is falling apart, I can’t take it anymore.' => Predicted Category: suicide
Text: 'I keep thinking about ways to make the pain stop.' => Predicted Category: suicide
Text: 'I’ve lost all hope, nothing matters anymore.' => Predicted Category: suicide
Text: 'It feels like the whole world is moving on without me.' => Predicted Category: suicide
Text: 'I had a great time with my friends today, feeling really happy!' => Predicted Category: non-suicide
Text: 'Im soooo proud about my self' => Predicted Category: non-suicide
Text: 'I failed my test today, but I will try harder next time.' => Pr

In [ ]:
import zipfile
import os

# Path to the zip files in Google Drive
emt_model_zip_path = '/content/drive/MyDrive/bert_emotion_model.zip'

# Directory to unzip the files
emt_model_dir = '/content/bert_emotion_model'

# Unzip the files
with zipfile.ZipFile(emt_model_zip_path, 'r') as zip_ref:
    zip_ref.extractall(emt_model_dir)

print("Emotion Model unzipped successfully!")

Emotion Model unzipped successfully!


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer

# Path to the unzipped model and tokenizer in Colab
emt_model_name = '/content/bert_emotion_model'  # Path to the unzipped model directory

# Load the TensorFlow model
emt_model = tf.saved_model.load(emt_model_name)

print("Emotion Model loaded successfully!")

Emotion Model loaded successfully!


In [ ]:
import tensorflow as tf

def test_emotion_detection(texts, emt_model, tokenizer, label_mapping):
    """
    Predicts the emotion of input texts.

    Args:
        texts (list): List of text strings to classify.
        emt_model: Trained BERT emotion classifier model.
        tokenizer: Tokenizer used during training.
        label_mapping (dict): Dictionary mapping label indices to emotion labels.

    Returns:
        None. Prints out the text and its predicted emotion.
    """
    # Tokenize input text, ensuring padding and truncation
    # Change max_length to 87 to match the model's expected input shape
    inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=87, return_tensors="tf", return_token_type_ids=True)

    # Cast input tensors to tf.int64
    inputs = {key: tf.cast(value, tf.int64) for key, value in inputs.items()}

    # Run inference using the model
    infer = emt_model.signatures['serving_default']
    outputs = infer(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], token_type_ids=inputs['token_type_ids'])

    # Get predictions
    logits = outputs['output_1']
    predictions = tf.argmax(logits, axis=1).numpy()

    # Map predictions to emotion labels
    predicted_emotions = [label_mapping[pred] for pred in predictions]

    # Print results
    print("\nPredicted Emotions:")
    for text, emotion in zip(texts, predicted_emotions):
        print(f"Text: '{text}' => Predicted Emotion: {emotion}")

# Example texts to classify
test_texts = [
    "I am so excited for my birthday party tomorrow!",
    "I feel so angry, they always take me for granted!",
    "Tired of crying my eyes out every night",
    "I think I might be falling in love, they make me so happy!",
    "Why is everyone against me? I feel so betrayed.",
    "Today was one of the best days ever, I feel amazing!",
    "I hate when people lie to me, it makes me so mad.",
    "I’m so nervous about my final exams, I hope I do well.",
    "I can’t stop smiling, today was just perfect!",
    "I feel like crying, everything is going wrong.",
    "Wow, I can’t believe I actually did it! So proud of myself.",
    "I’m shaking, that horror movie scared me so much!",
    "I just feel so alone even when I’m surrounded by people.",
    "I finally got the job! I can’t believe it, I’m so happy!",
    "Ugh, traffic was so bad today, I was so annoyed.",
    "I feel so peaceful watching the sunset, it’s beautiful."
]

# Define the label mapping
label_mapping = {
    1: "joy",
    3: "anger",
    0: "sadness",
    5: "surprise",
    2: "love",
    4: "fear"
}

# Run the test on the emotion model
test_emotion_detection(test_texts, emt_model, tokenizer, label_mapping)


Predicted Emotions:
Text: 'I am so excited for my birthday party tomorrow!' => Predicted Emotion: joy
Text: 'I feel so angry, they always take me for granted!' => Predicted Emotion: anger
Text: 'Tired of crying my eyes out every night' => Predicted Emotion: sadness
Text: 'I think I might be falling in love, they make me so happy!' => Predicted Emotion: joy
Text: 'Why is everyone against me? I feel so betrayed.' => Predicted Emotion: sadness
Text: 'Today was one of the best days ever, I feel amazing!' => Predicted Emotion: surprise
Text: 'I hate when people lie to me, it makes me so mad.' => Predicted Emotion: anger
Text: 'I’m so nervous about my final exams, I hope I do well.' => Predicted Emotion: fear
Text: 'I can’t stop smiling, today was just perfect!' => Predicted Emotion: joy
Text: 'I feel like crying, everything is going wrong.' => Predicted Emotion: fear
Text: 'Wow, I can’t believe I actually did it! So proud of myself.' => Predicted Emotion: joy
Text: 'I’m shaking, that horro

In [ ]:
import tensorflow as tf

def detect_suicide_and_emotion(texts, model, emt_model, tokenizer, label_mapping):
    """
    First detects if the text contains suicidal thoughts.
    If no suicide risk is detected, it then predicts the emotion of the text.

    Args:
        texts (list): List of text strings to classify.
        suicide_model: Trained BERT suicide detection model.
        emotion_model: Trained BERT emotion classification model.
        tokenizer: Tokenizer used during training.
        emotion_label_mapping (dict): Dictionary mapping label indices to emotion labels.

    Returns:
        None. Prints out the text and its detected category (suicide/non-suicide) and emotion.
    """
    suicide_label_mapping = {0: "non-suicide", 1: "suicide"}

    risk_levels = {"sadness": "Medium Risk", "anger": "Medium Risk", "fear": "Medium Risk"}

    # Process each text individually to handle different tokenization lengths
    for text in texts:
        # Tokenize input text for suicide detection
        inputs_suicide = tokenizer([text], padding='max_length', truncation=True, max_length=128, return_tensors="tf", return_token_type_ids=True)
        inputs_suicide = {key: tf.cast(value, tf.int64) for key, value in inputs_suicide.items()}

        # Run suicide detection model
        infer_suicide = model.signatures['serving_default']
        suicide_outputs = infer_suicide(input_ids=inputs_suicide['input_ids'], attention_mask=inputs_suicide['attention_mask'], token_type_ids=inputs_suicide['token_type_ids'])

        # Extract logits and make predictions
        suicide_logits = suicide_outputs['output_1']
        suicide_prediction = tf.argmax(suicide_logits, axis=1).numpy()[0]  # Get prediction for the single text

        suicide_label = suicide_label_mapping[suicide_prediction]

        if suicide_label == "suicide":
            print(f"Text: '{text}' => High Risk")
        else:
            # Tokenize input text for emotion detection with correct max_length
            inputs_emotion = tokenizer([text], padding='max_length', truncation=True, max_length=87, return_tensors="tf", return_token_type_ids=True)
            inputs_emotion = {key: tf.cast(value, tf.int64) for key, value in inputs_emotion.items()}

            # If no suicide risk, proceed with emotion detection
            infer_emotion = emt_model.signatures['serving_default']
            emotion_outputs = infer_emotion(input_ids=inputs_emotion['input_ids'], attention_mask=inputs_emotion['attention_mask'], token_type_ids=inputs_emotion['token_type_ids'])

            # Get predictions
            emotion_logits = emotion_outputs['output_1']
            emotion_prediction = tf.argmax(emotion_logits, axis=1).numpy()[0]  # Get prediction for the single text

            # Map prediction to emotion label
            emotion_label = label_mapping[emotion_prediction]

            # Determine risk level
            risk_level = risk_levels.get(emotion_label, "Low Risk")

            print(f"Text: '{text}' => **Emotion:** {emotion_label} => **{risk_level}**")

# Example texts to classify
test_texts = [
    "I don’t know what’s wrong with me, I feel empty all the time.",
    "I love my best friend so much, they always make me smile.",
    "It’s been a terrible day, I feel like I just want to disappear.",
    "I can’t stop crying, why does life have to be this hard?",
    "aww shes so cute.. todays date was amazinggg.",
    "I’m scared that things will never get better for me.",
    "I had so much fun at the party tonight, best night ever!",
    "I feel like I’m stuck in a loop, nothing ever changes.",
    "I was so sad, Cried alot when I first got to know that",
    "I finally spoke up about my problems, I hope things change.",
    "I can’t stop thinking about how much I hate myself.",
    "I had the best day with my friends, I feel so loved!",
    "I feel so lost, I don’t even know what I want anymore.",
    "I wish someone could understand how much I’m struggling.",
    "I was so anxious about my test, but I think I did well.",
    "I laughed so hard todayyy.. my friends are so hillarious"
]


# Define the emotion label mapping
label_mapping = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Run the detection function
detect_suicide_and_emotion(test_texts, model, emt_model, tokenizer, label_mapping)


Text: 'I don’t know what’s wrong with me, I feel empty all the time.' => High Risk
Text: 'I love my best friend so much, they always make me smile.' => **Emotion:** love => **Low Risk**
Text: 'It’s been a terrible day, I feel like I just want to disappear.' => High Risk
Text: 'I can’t stop crying, why does life have to be this hard?' => High Risk
Text: 'aww shes so cute.. todays date was amazinggg.' => **Emotion:** joy => **Low Risk**
Text: 'I’m scared that things will never get better for me.' => High Risk
Text: 'I had so much fun at the party tonight, best night ever!' => **Emotion:** joy => **Low Risk**
Text: 'I feel like I’m stuck in a loop, nothing ever changes.' => High Risk
Text: 'I was so sad, Cried alot when I first got to know that' => **Emotion:** sadness => **Medium Risk**
Text: 'I finally spoke up about my problems, I hope things change.' => High Risk
Text: 'I can’t stop thinking about how much I hate myself.' => High Risk
Text: 'I had the best day with my friends, I feel 